In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging

logging.basicConfig(level=logging.DEBUG)


In [7]:
from dataset.cfg_token_validator import CFGTokenValidator, GenerationState

state = GenerationState.initial()

validator = CFGTokenValidator()

print(validator.get_valid_tokens(state))
state = validator.advance_with_token("def", state)
print(validator.get_valid_tokens(state))
state = validator.advance_with_token(" ", state)
print(validator.get_valid_tokens(state))
state = validator.advance_with_token("abc", state)
print(validator.get_valid_tokens(state))
state = validator.advance_with_token("(", state)
print(validator.get_valid_tokens(state))

{'def'}
{' '}
{'<IDENTIFIER>'}
{'('}
{'<IDENTIFIER>'}


In [3]:
from dataset.cfg_parser import CFGParser, CFGTerminal

from pprint import pprint

parser = CFGParser()

pprint("Initial stack:")
pprint(parser.stack)

print("Initial valid next tokens:")
pprint(parser.valid_next_tokens())


terminals = [CFGTerminal.DEF, "function_name", CFGTerminal.LPAREN, CFGTerminal.RPAREN, CFGTerminal.COLON]

for terminal in terminals:
    print(f"Advancing with {terminal}")
    parser.advance(terminal)
    pprint("Current stack:")
    pprint(parser.stack)
    print("Valid next tokens:")
    pprint(parser.valid_next_tokens())


DEBUG:dataset.cfg_parser:Productions for CFGNonTerminal.PROGRAM: [['FUNCTION']]
DEBUG:dataset.cfg_parser:Checking if FUNCTION can start with CFGTerminal.DEF, resolved into CFGNonTerminal.FUNCTION
DEBUG:dataset.cfg_parser:First symbol is CFGNonTerminal: CFGNonTerminal.FUNCTION
DEBUG:dataset.cfg_parser:Checking if FUNCTION_HEADER can start with CFGTerminal.DEF, resolved into CFGNonTerminal.FUNCTION_HEADER
DEBUG:dataset.cfg_parser:First symbol is CFGNonTerminal: CFGNonTerminal.FUNCTION_HEADER
DEBUG:dataset.cfg_parser:Checking if def can start with CFGTerminal.DEF, resolved into CFGTerminal.DEF
DEBUG:dataset.cfg_parser:First symbol is CFGTerminal: CFGTerminal.DEF
DEBUG:dataset.cfg_parser:Comparing CFGTerminal.DEF == CFGTerminal.DEF = True
DEBUG:dataset.cfg_parser:Productions for CFGNonTerminal.FUNCTION: [['FUNCTION_HEADER', 'NEWLINE', 'FUNCTION_BODY']]
DEBUG:dataset.cfg_parser:Checking if FUNCTION_HEADER can start with function_name, resolved into CFGNonTerminal.FUNCTION_HEADER
DEBUG:datas

'Initial stack:'
deque([<CFGNonTerminal.PROGRAM: 'PROGRAM'>])
Initial valid next tokens:
{<CFGTerminal.DEF: 'def'>}
Advancing with CFGTerminal.DEF
'Current stack:'
deque([<CFGNonTerminal.PROGRAM: 'PROGRAM'>,
       <CFGNonTerminal.FUNCTION: 'FUNCTION'>])
Valid next tokens:
{<CFGTerminal.DEF: 'def'>}
Advancing with function_name


ValueError: Token function_name not valid for nonterminal CFGNonTerminal.FUNCTION

In [ ]:
parser.stack

deque([<CFGNonTerminal.PROGRAM: 'PROGRAM'>,
       <CFGNonTerminal.FUNCTION: 'FUNCTION'>])